## Evaluation Output Format

```csv
# csv
message,accuracy,label
example message,0.8,1
```

## FLAN-T5

## CodeGen

## CodeTrans

- [Paper](https://arxiv.org/abs/2104.02443)
- [GitHub](https://github.com/agemagician/CodeTrans)
- [HuggingFace](https://huggingface.co/models?search=code_trans)
- [Datasets](https://www.dropbox.com/sh/mzxa2dq30gnot29/AABIf7wPxH5Oe0PZHJ5jPV22a?dl=0)

### Overview

- Released 2023
- Architecture
  - Encoder-decoder (based on T5)
- Learning Objective
  - Text-to-text
- Variants
  - Function Documentation Generation (Python, Java, Go, Php, Ruby, JavaScript)
    - [CodeSearchNet Corpus Collection](https://github.com/github/CodeSearchNet) Dataset
  - Source Code Summarization (Python, SQL, C#)
    - [CODENN StackOverflow](https://github.com/sriniiyer/codenn) Dataset
  - Code Comment Generation (Java only)
    - [DeepCom](https://github.com/xing-hu/DeepCom) Dataset
  - Commit Message Generation (Java only)
    - [CommitGen](https://sjiang1.github.io/commitgen) Dataset
  - API Sequence Recommendation (Java only)
    - [Deep API Learning](https://github.com/guxd/deepAPI) Dataset
  - Programming Language and Synthesis (LISP only)
    - [AlgoLisp](https://github.com/nearai/program_synthesis/tree/master/program_synthesis/algolisp) Dataset

### Pros

- Multiple Variants Trained to Perform Specific Tasks
- Source Code Summarization Model Trained on C#
- Transformer-based Encoder-Decoder Model
  - Good for Sequence-to-Sequence Tasks (e.g. Summarization, Translation)
- Uses and Evaluates Different Training Stratagies
  - Single-Task Learning, Transfer Learning, Multi-Task Learning, Multi-Task Learning with Fine-Tuning

### Cons

- **Not Trained on C dataset**
- **Not Trained to Perform Masked Language Modeling**
  - Needed for Accurate Text Classification
- No Longer State-of-the-Art
- **Model is Largely Abandoned**
  - Incomplete README, last Git commit on 1 June 2021
- Requires more work to prepare data: text-to-text format


## CodeBert